In [247]:
!pip install -q -U kaggle_environments

In [248]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

Агент всегда играет камень.

In [249]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Агент всегда играет бумага.

In [250]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1


Overwriting paper_agent.py


Агент всегда играет ножницы.

In [251]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


Агент повторяет прошлые действия оппонента. 

In [252]:
%%writefile copy_agent.py

import random
def copy_agent(observation, configuration):
    # Получает прошлое действие оппонента
    if observation.step > 0:
        return observation.lastOpponentAction
    # Первый шаг оппонента - случайное значение
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_agent.py


Агент возвращает действие, которое победит самое популярное из прошлых действий оппонента. 

In [253]:
%%writefile popular_agent.py

import random
# Прошлые действия оппоннета
opponent_actions = []

def popular_agent(observation, configuration):
    global opponent_actions
    
    count = len(opponent_actions)
    
    # Первый шаг оппонента - случайное значение
    if count == 0:
        opponent_actions.append(random.randrange(0, configuration.signs)) 
    # Находим самое используемое действие опонента из прошлых
    else:
        opponent_actions.append(observation.lastOpponentAction)
        action_count = [opponent_actions.count(0), opponent_actions.count(1), opponent_actions.count(2)]        
        popular_opponent_action = action_count.index(max(action_count))

        if popular_opponent_action == 0: 
            return 1
        elif popular_opponent_action == 1:
            return 2
        else:
            return 0

Overwriting popular_agent.py


Агент при победе в прошлой игре возвращает прошлое действие, при проигрыше или игре в ничью меняет действие на случайное, за исключением невыигрышного действия.

In [254]:
%%writefile last_agent.py

import random
last_my_action = None

def last_agent(observation, configuration):
    global last_my_action

    # Определение исхода игры
    def get_result(last_my_action, last_opponent_action):
        if last_my_action == last_opponent_action:
            return 0
        elif (last_my_action == 1 and last_opponent_action == 0) \
            or (last_my_action == 0 and last_opponent_action == 1) \
            or (last_my_action == 2 and last_opponent_action == 1):
            return 1
        else:
            return -1    

    if observation.step == 0:
    # Первый шаг - камень
        last_opponent_action = 0 
        last_my_action = 0
    else:
        last_opponent_action = observation.lastOpponentAction
    result = get_result(last_my_action, last_opponent_action)

    if result != 1: 
        # Если в прошлой игре не было победы, меняем действие на случайное, за исключением невыигрышного действия
        list_of_action = [action for action in [0, 1, 2] if action != last_my_action] 
        last_my_action = random.choice(list_of_action)       
    return last_my_action
    

Overwriting last_agent.py


Агент возвращает случайное значение. 

In [255]:
%%writefile random_agent.py

import random

def random_agent(observation, configuration):
    return random.randint(0, 2)

Overwriting random_agent.py


Агент, который возвращает свои предыдущие шаги cо сдвигом.

In [256]:
%%writefile my_previous_agent.py

# Действие прошлых игр
last_games = []

def my_previous_agent(observation, configuration):
    global last_games

    if observation.step == 0:
    # Первый шаг - камень
        agent = 0
        last_games.append(agent)
    else:
        agent = (last_games[-1] + 1) % 3 
        last_games.append(agent)
    return agent

Overwriting my_previous_agent.py


Абонент, который возвращает предыдущие шаги оппонента cо сдвигом

In [257]:
%%writefile opponent_previous_agent.py


# Действие оппонента в прошлых играх
last_games = []

def opponent_previous_agent(observation, configuration):
    global last_games

    if observation.step == 0:
    # Первый шаг оппонента - камень    
        opponent_action = 0
        last_games.append(opponent_action)
    else:
        last_games[-1] = observation.lastOpponentAction
        opponent_action = (last_games[-1] + 1) % 3
        last_games.append(opponent_action)
    return opponent_action

Overwriting opponent_previous_agent.py


Абонент возвращает камень, если шаг четный, случайное значение среди ножниц и бумаги - если шаг нечетный.

In [258]:
%%writefile even_agent.py

import random
def even_agent(observation, configuration):  
    if observation.step % 2 == 0:
        return 0
    else:
        return random.choice([1, 2])

Overwriting even_agent.py


Агент возвращает действие, которое победит самое популярное из прошлых действий оппонента, если вероятность выпадания самого популярного действия оппонета больше 20%. 
В противном случае агент возвращает самое популярное из прошлых действий оппонента.

In [259]:
%%writefile probability_agent.py

import random

# Прошлые действия оппонента
opponent_actions = []

def probability_agent(observation, configuration):
    global opponent_actions
    
    count = len(opponent_actions) # Количество прошлых действий 
    
    # Первый шаг оппонента - случайное значение.
    if count == 0:
        opponent_actions.append(random.randrange(0, configuration.signs)) 
    # Находим самое используемое действие опонента из прошлых.
    else:
        opponent_actions.append(observation.lastOpponentAction)
        action_count = [opponent_actions.count(0), opponent_actions.count(1), opponent_actions.count(2)]        
        popular_opponent_action = action_count.index(max(action_count))
        # Вероятность выпадания популярного действия. 
        probability = popular_opponent_action / count 

        if probability > 20:
            if popular_opponent_action == 0: 
                return 1
            elif popular_opponent_action == 1:
                return 2
            else:
                return 0
        else:
            return popular_opponent_action

Overwriting probability_agent.py


Агент возвращает следующее по отношению к действиям оппонента действие. Порядок действий задан следующим образом: камень, бумага, ножницы.

In [260]:
%%writefile ordered_agent.py

import random

def ordered_agent(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return 1
        elif observation.lastOpponentAction == 1:
            return 2
        else:
            return 0
    else:
        # Первый шаг - случайное действие
        return random.randrange(0, configuration.signs)

Overwriting ordered_agent.py


Агент при победе возвращает действие, которое не играло в прошлой игре, при проигрыше или игре в ничью - действие оппонента в прошлой игре. 

In [261]:
%%writefile thirteen_agent.py

# Действия игроков в прошлых играх
last_games = []

def thirteen_agent(observation, configuration):
    global last_games

    # Определение исхода игры
    def get_result(last_my_action, last_opponent_action):
        if last_my_action == last_opponent_action:
            return 0
        elif (last_my_action == 1 and last_opponent_action == 0) \
            or (last_my_action == 0 and last_opponent_action == 1) \
            or (last_my_action == 2 and last_opponent_action == 1):
            return 1
        else:
            return -1  

    if observation.step == 0:
    # Первый шаг - камень
        agent = 0
        last_games.append({'agent' : agent, 'opponent' : None})
        return agent
    else:
        last_games[-1]['opponent'] = observation.lastOpponentAction
        result = get_result(last_games[-1]['agent'], last_games[-1]['opponent']) 
        
        if result == 1: 
            agent = [action for action in [0, 1, 2] if action != last_games[-1]['agent'] and action != last_games[-1]['opponent']] 
            last_games.append({'agent' : agent[0], 'opponent': None})
        else: 
            agent = last_games[-1]['opponent'] 
            last_games.append({'agent' : agent, 'opponent': None})
        return last_games[-2]['agent']
            

Overwriting thirteen_agent.py


Составим турнирную таблицу, отображающую количество побед, проигрышей и игр в ничью агентов.
Для формирования турнирной таблицы воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

Подготовим списки агентов для игр, выходной словарь с результатами игр и функцию определения результатов игр.

In [262]:
# Cписок всех агентов
agents = [
    "rock_agent",
    "paper_agent", 
    "scissors_agent",
    "copy_agent",
    "popular_agent", 
    "last_agent",
    "random_agent", 
    "my_previous_agent",
    "opponent_previous_agent",
    "even_agent", 
    "probability_agent",
    "ordered_agent",
    "thirteen_agent"
    ]
 
list_agents = [agent_name + ".py" for agent_name in agents]


# Словарь с результатами игр по каждому агенту
output = {}
for agent in list_agents:
    output[agent] = {
        'win'  : 0,
        'loss' : 0,
        'tie'  : 0
    }

# Функция определение результата игры
def get_result(agent1, agent2):
    output = evaluate(
                "rps", 
                [agent1, agent2], 
                configuration={"episodeSteps": 100})
    return agent1, agent2, output[0]

Проведем турнир между агентами, считая, что агент не может играть с таким же агентом как и он. 

In [263]:
results = []
for i in range(len(list_agents)):
    for j in range(len(list_agents)):
        if i != j:
            results.append(get_result(list_agents[i], list_agents[j]))

Подсчитаем количество побед, проигрышей и игр в ничью по каждому агенту.

In [264]:
for result in results:
    if abs(result[2][0]) < 20:
        output[result[0]]['tie']  += 1
        output[result[1]]['tie']  += 1
    elif result[2][0] >= 20:
        output[result[0]]['win']  += 1
        output[result[1]]['loss'] += 1
    elif result[2][0] <= -20:
        output[result[0]]['loss'] += 1
        output[result[1]]['win']  += 1

# Сортировка словаря с результатами игр по количеству побед
sorted_output = dict(sorted(output.items(), key=lambda x: x[1].get('win'), reverse=True))

Выведем полученные результаты в виде таблицы.

In [265]:
# определим отступы для вывода
shift = max([len(name_agent) for name_agent in list_agents])
little_shift = 6

# заголовок для удобства
print('Name agent'.ljust(shift), 'Win'.ljust(little_shift), 'Loss'.ljust(little_shift), 'Tie')

# вывод турнирной таблицы, отсортированной по количеству побед
for key, value in sorted_output.items():
    print(key.ljust(shift), str(value['win']).ljust(little_shift), str(value['loss']).ljust(little_shift), value['tie'])

Name agent                 Win    Loss   Tie
popular_agent.py           12     6      6
opponent_previous_agent.py 12     4      8
ordered_agent.py           11     5      8
thirteen_agent.py          9      0      15
even_agent.py              8      6      10
my_previous_agent.py       6      3      15
last_agent.py              5      3      16
paper_agent.py             4      10     10
rock_agent.py              2      10     12
scissors_agent.py          2      10     12
copy_agent.py              1      7      16
random_agent.py            0      0      24
probability_agent.py       0      8      16
